# One dimensional line search via estimation

A robust optimization technique that assumes the function to be optimized is continuous, but not differentiable, and noisy. The function is approximated by a parabola over the set of points that have been computed, assuming that the function is convex withn the range of points

In [121]:
# Example of optimization by bisection line search
import math
import bisect
import numpy as np

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [122]:
# Initialization 
range_min, range_max = (-1,1)
# initial_guess = 0.5
# initial_step_size = 0.1
# epsilon = 1.0e-6

# tuples of (x, f(x)), ordered by x.
search_grid = []

In [123]:
def f(x):
    'Function over the range to minimize in one dim'
    sc = 6.0
    return sc*sc*math.exp((x-1.5)*sc) + sc*sc*math.exp(-(1.9 + x)*sc) + 0.1*np.random.random_sample()

def add_f_to_grid(x):
    'memoization of the points searched'
    global search_grid
    # Binary search would be better
    found_k = [k for k in range(len(search_grid)) if abs(x-search_grid[k][0]) <= epsilon]
    if found_k == []:
        new_pt = (x, f(x))
        bisect.insort_left(search_grid, new_pt)
        return new_pt
    else:
        return search_grid[found_k[0]]
    
def init_grid(no_pts=3, shrinkage=3.0):
    global search_grid
    half_range = (range_max - range_min)/2
    grid_x = np.linspace(range_min + half_range/shrinkage, range_max - half_range/shrinkage, no_pts)
    search_grid = list(zip(grid_x, map(f, grid_x)))

In [124]:
# Create some starting points. 
init_grid(6)

In [125]:
# This is how to create more points
for pt in np.linspace(-0.3, 0.5, 6):
   add_f_to_grid(pt)
search_grid

[(-0.6666666666666667, 0.10687945479928289),
 (-0.4, 0.1025807485630554),
 (-0.3, 0.05596342826479847),
 (-0.13999999999999999, 0.03924077073025912),
 (-0.1333333333333333, 0.006715753357591219),
 (0.020000000000000018, 0.030809368131613536),
 (0.13333333333333341, 0.10449945683506305),
 (0.18, 0.020163492101513653),
 (0.34, 0.13133211791690075),
 (0.40000000000000013, 0.11648912850308621),
 (0.5, 0.1891739115391955),
 (0.6666666666666667, 0.2695427304539337)]

In [126]:
# Use a quadratic regression to estimate the minimum of the function
# By centering the data around 0 the columns of the design matrix form an orthogonal basis function
def dm_sample(the_pt):
    'From an x,y tuple build a design matrix row'
    x = the_pt[0]
    return (1, x, x*x)

def points_design_matrix():
    'The quadratic regression design matrix'
    dm = np.empty((0,3),dtype='float')
    for row in search_grid:
        dm = np.vstack((dm, np.array(dm_sample(row), dtype='float')))
    return dm

In [127]:
X = points_design_matrix()
y = [z[1] for z in search_grid]

# The fit to the search grid points
# For outputs, see https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.linalg.lstsq.html
parabola_fit = np.linalg.lstsq(X,y, rcond=-1)
parabola_fit

(array([0.04173243, 0.09934442, 0.34979398]),
 array([0.00897678]),
 3,
 array([3.50679241, 1.31088128, 0.51839461]))

In [128]:
def quadratic(pts, coeffs):
    'Compute the fitted values for the parabola fit'
    c = coeffs[0]
    b = coeffs[1]
    a = coeffs[2]
    y = [a * x * x + b * x + c for x in pts]
    # Compute estimated minimum
    print('Estimated min at: {}'.format(-0.5*b/a))
    return y

x = [z[0] for z in search_grid]
parabola_pts = (x, quadratic(x, parabola_fit[0]))

Estimated min at: -0.14200419281627252


In [129]:
def residual_errors(p_pts, y):
    errs = [abs(pt[1] - pt[0]) for pt in zip(y, p_pts[1])]
    return errs
    
print('Regression residual errors: {}'.format(sum(residual_errors(parabola_pts, y))/len(y)))

Regression residual errors: 0.022809818454043115


In [130]:
def plot_search_grid(parabola_pts):
    p = figure(plot_width = 600, plot_height = 600, 
           title = 'Current points',
           x_axis_label = 'x', y_axis_label = 'f(x)')
    pts = list(zip(*search_grid))
    p.circle(pts[0], pts[1],  color = 'darkred', size=6)
    # Add a parabola approx.
    p.line(parabola_pts[0], parabola_pts[1], color = 'lightblue')
    show(p)
    
# Show both the search points and the best parabolic fit
plot_search_grid(parabola_pts)